<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/archit/BERT_NER_with_Dice_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==2.6.0
!pip install seqeval
!pip install urllib3 --upgrade
!pip install sadice

  Using cached https://files.pythonhosted.org/packages/56/aa/4ef5aa67a9a62505db124a5cb5262332d1d4153462eb8fd89c9fa41e5d92/urllib3-1.25.11-py2.py3-none-any.whl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfully uninstalled urllib3-1.26.2
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


In [ ]:
import sys
import time
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext import data
import transformers
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

from sadice import SelfAdjDiceLoss

torch.manual_seed(12)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
train_path = '/content/drive/My Drive/ToxicSpan_CS669V/BERT_Preprocess/train.pkl'
val_path = '/content/drive/My Drive/ToxicSpan_CS669V/BERT_Preprocess/val.pkl'

with open(train_path, 'rb') as f:
  train_data = pickle.load(f)
  f.close()

with open(val_path, 'rb') as f:
  val_data = pickle.load(f)
  f.close()

In [ ]:
val_data

In [ ]:
MAX_LEN = 500
BATCH_SIZE = 8

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case = False)

In [ ]:
X_train = train_data['token_final']
X_val = val_data['token_final']
Y_train = train_data['target_final']
Y_val = val_data['target_final']

In [ ]:
CLASSES = {'0':0, '1':1, '[PAD]':2}

In [ ]:
def tokenize_bert(x, y):
  sentence = []
  labels = [0]
  for word, label in zip(x, y):
    tokenized_word = tokenizer.tokenize(word)
    sentence.extend(tokenized_word)
    labels.extend([label for i in range(len(tokenized_word))])
  labels.append(0)
  return(sentence, labels)

In [ ]:
len_train = len(X_train)
len_val = len(X_val)

for i in range(len_train):
  X_train[i], Y_train[i] = tokenize_bert(X_train[i], Y_train[i])

for i in range(len_val):
  X_val[i], Y_val[i] = tokenize_bert(X_val[i], Y_val[i])

In [ ]:
X_train_id = pad_sequences([tokenizer.encode(text) for text in X_train], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_train_id = pad_sequences(Y_train, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')
X_val_id = pad_sequences([tokenizer.encode(text) for text in X_val], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_val_id = pad_sequences(Y_val, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')

In [ ]:
def get_attention_mask(x):
  return([[(i!=0) for i in text] for text in x])

In [ ]:
attention_mask_train = get_attention_mask(X_train_id)
attention_mask_val = get_attention_mask(X_val_id)

In [ ]:
X_train_id = torch.tensor(X_train_id)
Y_train_id = torch.tensor(Y_train_id)
X_val_id = torch.tensor(X_val_id) 
Y_val_id = torch.tensor(Y_val_id)
attention_mask_train = torch.tensor(attention_mask_train)
attention_mask_val = torch.tensor(attention_mask_val)

In [ ]:
train_data = TensorDataset(X_train_id, attention_mask_train, Y_train_id)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(X_val_id, attention_mask_val, Y_val_id)
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [ ]:
model = BertForTokenClassification.from_pretrained(
    'bert-base-cased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda()

In [ ]:
FINE_TUNING = True
if FINE_TUNING:
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
                                  {'params' : [p for n,p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate' : 0.01},
                                  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]
else:
  param_optimizer = list(model.classifier.named_parameters())
  optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)

In [ ]:
epochs = 10
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
criterion = SelfAdjDiceLoss(alpha=0.8, gamma=1)
criterion = criterion.to(device)

In [ ]:
def get_text_lengths(masks):
  lengths = []
  for mask in masks:
    lengths.append(torch.sum(mask).item())
  return(lengths)

In [ ]:
train_loss, val_loss = [], []
train_acc, val_acc = [], []
train_f1, val_f1 = [], []

l = 100

for epoch in trange(epochs, desc = 'Epoch'):
  print(epoch)
  model.train()
  t_loss, t_acc = 0, 0
  predictions, true_labels = [], []
  for step, batch in enumerate(train_dataloader):
    model.zero_grad()
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels = b_labels)
    # loss = outputs[0]
    active_logits = torch.masked_select(outputs[1], b_input_mask.unsqueeze(-1).repeat(1, 1, 2))
    active_labels = torch.masked_select(b_labels, b_input_mask)
    # print(active_labels.shape, active_labels)
    loss = criterion(active_logits.view(-1, 2), active_labels)
    loss.backward()
    t_loss += loss.item()
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    optimizer.step()
    scheduler.step()
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

    
  print(f"Train Loss : {t_loss/len(train_dataloader)}")
  train_loss.append(t_loss/len(train_dataloader))
  pred_tags = [p_i for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if l_i != 2]
  valid_tags = [l_i for l in true_labels
                                for l_i in l if l_i != 2]
  train_acc.append(accuracy_score(pred_tags, valid_tags))
  train_f1.append(f1_score(pred_tags, valid_tags))
  print("Train Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Train F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  print()

  model.eval()
  v_loss, v_accuracy = 0, 0
  predictions , true_labels = [], []
  for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    
    active_logits = torch.masked_select(outputs[1], b_input_mask.unsqueeze(-1).repeat(1, 1, 2))
    active_labels = torch.masked_select(b_labels, b_input_mask)
    # print(active_labels.shape, active_labels)
    loss = criterion(active_logits.view(-1, 2), active_labels)
    
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    v_loss += loss.item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
    
  
  v_loss = v_loss/len(val_dataloader)
  val_loss.append(v_loss)
  if(v_loss < l):
    l = v_loss
    print("Model Checkpoint")
  torch.save(model, f'/content/drive/My Drive/model{epoch}.pt')
    
  print(f"Validation Loss : {v_loss}")
  pred_tags = [p_i for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if l_i != 2]
  valid_tags = [l_i for l in true_labels
                                for l_i in l if l_i != 2]
  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  val_acc.append(accuracy_score(pred_tags, valid_tags))
  val_f1.append(f1_score(pred_tags, valid_tags))
  print()

In [ ]:
conf = confusion_matrix(valid_tags, pred_tags)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(conf, range(2), range(2))
# plt.figure(figsize=(10,7))
sns.set(font_scale=1.4) # for label size
sns.heatmap(df_cm, annot=True, annot_kws={"size": 12}, fmt='d') # font size

plt.show()
# plt.savefig('bert.pdf')

In [ ]:
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_Loss':train_loss,'Val_Loss': val_loss}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,0.5),xlabel='No. of epochs',ylabel='Loss',title='BERT')
ax.set_xticks(range(5)) # <--- s
ax.set_xticklabels(['1','2','3','4', '5'])

In [ ]:
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_Acc':train_acc,'Val_Acc': val_acc}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,1),xlabel='No. of epochs',ylabel='Acc',title='BERT')
ax.set_xticks(range(5)) # <--- s
ax.set_xticklabels(['1','2','3','4', '5'])

In [ ]:
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_F1_Score':train_f1,'Val_F1_Score': val_f1}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,1),xlabel='No. of epochs',ylabel='F1 Score',title='BERT')
ax.set_xticks(range(10)) # <--- s
ax.set_xticklabels([str(i) for i in range(10)])